### Testing Hyper-param tuning with Ray and Tune on a Simple CNN.
https://docs.ray.io/en/latest/tune/examples/tune-pytorch-cifar.html

In [1]:
# Ensure the necessary imports
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler

from src.engine import simpleCNN_engine
from src.model_simpleCNN import SimpleCNN
from src.dataset import SignalsDataset
from src.preprocess import Prep_data_for_CNN
from src.hyperparam import RayTuning

In [2]:
import pandas as pd
import numpy as np

In [3]:
windowed_df = Prep_data_for_CNN().get_window_df_from_dir('./data/sample_sub_segmented_data_112.csv',
                                                         prep_for_FMAPextract=False)

In [4]:
X, y, uniq = Prep_data_for_CNN().prep_input_for_CNN(windowed_df)

In [5]:
uniq, np.unique(y)

(array(['Not_driving', 'User1', 'User2'], dtype='<U11'),
 array([0, 1, 2], dtype=int64))

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

In [7]:
# Perform a train-test split
np.random.seed(1)
X_tr, X_test, y_tr, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_tr, y_tr, test_size=0.2, stratify=y_tr)

class_weights = torch.tensor(compute_class_weight('balanced', classes=np.unique(y_train), y=y_train), dtype=torch.float32)
class_weights = class_weights.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

# normalizing X
mean = X_train.mean(axis=(0, 2), keepdims=True)
std = X_train.std(axis=(0, 2), keepdims=True)

X_train = (X_train - mean) / std
X_valid = (X_valid - mean) / std
X_test = (X_test - mean) / std

In [8]:
# Create datasets for each feature
feature_indices = {'long_acc': 0, 'tranv_acc': 1, 'ang_vel': 2}

train_datasets = {feature: SignalsDataset(X_train, y_train, idx) for feature, idx in feature_indices.items()}
valid_datasets = {feature: SignalsDataset(X_valid, y_valid, idx) for feature, idx in feature_indices.items()}
test_datasets = {feature: SignalsDataset(X_test, y_test, idx) for feature, idx in feature_indices.items()}

## dataset for extracting feature maps from data
all_dataset = {feature: SignalsDataset((X-mean)/std, y, idx) for feature, idx in feature_indices.items()}

In [9]:
np.random.seed(1)

batch_size = 8 
train_dl_long = DataLoader(train_datasets['long_acc'], batch_size, shuffle=True)
train_dl_tranv = DataLoader(train_datasets['tranv_acc'], batch_size, shuffle=True)
train_dl_angvel = DataLoader(train_datasets['ang_vel'], batch_size, shuffle=True)

valid_dl_long = DataLoader(valid_datasets['long_acc'], batch_size, shuffle=False)
valid_dl_tranv = DataLoader(valid_datasets['tranv_acc'], batch_size, shuffle=False)
valid_dl_angvel = DataLoader(valid_datasets['ang_vel'], batch_size, shuffle=False)

test_dl_long = DataLoader(test_datasets['long_acc'], batch_size, shuffle=False)
test_dl_tranv = DataLoader(test_datasets['tranv_acc'], batch_size, shuffle=False)
test_dl_angvel = DataLoader(test_datasets['ang_vel'], batch_size, shuffle=False)

Define Hyperparameter search space

In [10]:
# Enhanced Hyperparameter search space
config = {
    "optimizer": tune.choice(["adam", "sgd"]),
    "lr": tune.loguniform(1e-5, 1e-1),
    "scheduler": tune.choice(["step", "exp", "cos"]),
    "step_size": tune.choice([10, 20, 30, 40, 50]),
    "gamma": tune.uniform(0.1, 0.9),
    "weight_decay": tune.choice([0.0, 1e-4, 1e-3, 1e-2]),
    "epochs": tune.choice([10, 20, 30]),
    "batch_size": tune.choice([8, 16, 32, 64])
}

criterion = nn.CrossEntropyLoss(weight=class_weights)
save_dir = 'test'
model = SimpleCNN
modelType = 'SimpleCNN'
engine = simpleCNN_engine

Define a training function that integrates with Ray Tune

In [11]:
mytuner = RayTuning(config, save_dir, criterion, model, modelType, engine)

In [12]:
from IPython.display import display, HTML

# Load custom CSS file
css = HTML('<link rel="stylesheet" type="text/css" href="./custom.css">')  # Ensure the path is correct
display(css)

In [13]:
mytuner.main(train_datasets['long_acc'], valid_datasets['long_acc'], test_datasets['long_acc'])

(train_model pid=12104) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/test/trial_1d5b1_00000/checkpoint_000000)
(train_model pid=12104) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/test/trial_1d5b1_00000/checkpoint_000001)
(train_model pid=12104) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/test/trial_1d5b1_00000/checkpoint_000002)
(train_model pid=12104) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspac

Best trial config: {'optimizer': 'sgd', 'lr': 1.3698885785105936e-05, 'scheduler': 'cos', 'step_size': 10, 'gamma': 0.6985325235038715, 'weight_decay': 0.01, 'epochs': 30, 'batch_size': 64}
Best trial final validation loss: 1.0962030291557312
Best trial final validation accuracy: 44.9275%
Test Accuracy: 0.4418604651162791
Best trial test set accuracy: 0.4419%


Go with adam, exp, batchsize=32


In [14]:
# Enhanced Hyperparameter search space
config = {
    "optimizer": tune.choice(["adam"]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "scheduler": tune.choice(["exp"]),
    #"step_size": tune.choice([10, 20, 30, 40, 50]),
    "gamma": tune.uniform(0.1, 0.9),
    #"weight_decay": tune.choice([0.0, 1e-4, 1e-3, 1e-2]),
    "epochs": tune.choice([30]),
    "batch_size": tune.choice([16])
}

In [2]:
1e-4

0.0001

In [15]:
mytuner = RayTuning(config, save_dir, criterion, model, modelType, engine)

In [16]:
mytuner.main(train_datasets['long_acc'], valid_datasets['long_acc'], test_datasets['long_acc'])

(train_model pid=4416) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/test/trial_eea09_00000/checkpoint_000000)
(train_model pid=4416) Checkpoint successfully created at: Checkpoint(filesystem=local, path=c:/Users/LEGION/Documents/Msc Data Science - Uni of Exeter/ECMM451 - Data Science Research Project (2023)/workspace/ray_results/test/trial_eea09_00000/checkpoint_000001)


: 